In [3]:
# import the required packages
import pandas as pd
import pyspark
import numpy as np

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType, DoubleType
from pyspark.sql.functions import col, trim, translate, to_date, when, split, explode
spark = SparkSession.builder.appName('google').getOrCreate()

In [5]:
# load the data using pandas
playstoredf = pd.read_csv('data\googleplaystore.csv')
reviewsdf = pd.read_csv('data\googleplaystore_user_reviews.csv')

playstoredf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10841 entries, 0 to 10840
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   App             10841 non-null  object 
 1   Category        10841 non-null  object 
 2   Rating          9367 non-null   float64
 3   Reviews         10841 non-null  object 
 4   Size            10841 non-null  object 
 5   Installs        10841 non-null  object 
 6   Type            10840 non-null  object 
 7   Price           10841 non-null  object 
 8   Content Rating  10840 non-null  object 
 9   Genres          10841 non-null  object 
 10  Last Updated    10841 non-null  object 
 11  Current Ver     10833 non-null  object 
 12  Android Ver     10838 non-null  object 
dtypes: float64(1), object(12)
memory usage: 1.1+ MB


In [6]:
playstoredf

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10836,Sya9a Maroc - FR,FAMILY,4.5,38,53M,"5,000+",Free,0,Everyone,Education,"July 25, 2017",1.48,4.1 and up
10837,Fr. Mike Schmitz Audio Teachings,FAMILY,5.0,4,3.6M,100+,Free,0,Everyone,Education,"July 6, 2018",1.0,4.1 and up
10838,Parkinson Exercices FR,MEDICAL,NaN,3,9.5M,"1,000+",Free,0,Everyone,Medical,"January 20, 2017",1.0,2.2 and up
10839,The SCP Foundation DB fr nn5n,BOOKS_AND_REFERENCE,4.5,114,Varies with device,"1,000+",Free,0,Mature 17+,Books & Reference,"January 19, 2015",Varies with device,Varies with device


In [7]:
# load data using pyspark
store = spark.read.csv('data/googleplaystore.csv', sep=',',
                         inferSchema=True, header=True)
reviews = spark.read.csv('data/googleplaystore_user_reviews.csv',sep=',',
                         inferSchema=True, header=True)

In [8]:
type(store)
type(reviews)

# demo convert pyspark data frame to pandas data frame
df = store.toPandas()
type(df)

pandas.core.frame.DataFrame

In [9]:
# get the schema of the dataframe
store.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)



In [10]:
# cast the column with float
store = store.withColumn("Rating",store["Rating"].cast(DoubleType()))

store.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)



In [11]:
store.show()

+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+------------------+------------------+------------+
|                 App|      Category|Rating|Reviews|Size|   Installs|Type|Price|Content Rating|              Genres|      Last Updated|       Current Ver| Android Ver|
+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+------------------+------------------+------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 19M|    10,000+|Free|    0|      Everyone|        Art & Design|   January 7, 2018|             1.0.0|4.0.3 and up|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967| 14M|   500,000+|Free|    0|      Everyone|Art & Design;Pret...|  January 15, 2018|             2.0.0|4.0.3 and up|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|  87510|8.7M| 5,000,000+|Free|    0|      Everyone|        Art & Design|    August 1, 2018|             1.2.4|4.0.3 

In [12]:
#count rows
store.count()

10841

In [13]:
#count distinct rows for App column
store.select("App").distinct().count()

9660

In [14]:
# creating a temporary view of 
# Dataframe and storing it into storeTemp
store.createOrReplaceTempView("storeTemp")
 
# using the SQL query to count all non distinct app
spark.sql("select App,count(*) as total from storeTemp group by App having total>1").show() 
# using the SQL query to count all non distinct app
spark.sql("select count(*) from (select App,count(*) as total from storeTemp group by App having total>1) as a").show() 

+--------------------+-----+
|                 App|total|
+--------------------+-----+
|Google Chrome: Fa...|    3|
|free video calls ...|    3|
|       Magic Tiles 3|    4|
|       Nova Launcher|    2|
|Live Talk - Free ...|    2|
|No.Draw - Colors ...|    2|
|Cricbuzz - Live C...|    2|
|        Google Earth|    2|
|Spanish English T...|    2|
|               Slack|    3|
|        Sway Medical|    2|
|Viki: Asian TV Dr...|    3|
|Strawberry Shortc...|    3|
|Medical ID - In C...|    2|
|FilterGrid - Cam&...|    2|
|Weight Loss Runni...|    3|
|      Bubble Shooter|    6|
|Photo Editor Self...|    2|
|To-Do Calendar Pl...|    2|
|Food Calorie Calc...|    2|
+--------------------+-----+
only showing top 20 rows

+--------+
|count(1)|
+--------+
|     798|
+--------+



In [44]:
# created dim app df by selecting columns from store df
dim_app = store.select(col("App"),col("Current Ver"),col("Android Ver"),col("Last Updated"),col("Size"),col("Content Rating"),col("Category"))
type(dim_app)

pyspark.sql.dataframe.DataFrame

In [45]:
dim_app.show()

+--------------------+------------------+------------+------------------+----+--------------+--------------+
|                 App|       Current Ver| Android Ver|      Last Updated|Size|Content Rating|      Category|
+--------------------+------------------+------------+------------------+----+--------------+--------------+
|Photo Editor & Ca...|             1.0.0|4.0.3 and up|   January 7, 2018| 19M|      Everyone|ART_AND_DESIGN|
| Coloring book moana|             2.0.0|4.0.3 and up|  January 15, 2018| 14M|      Everyone|ART_AND_DESIGN|
|U Launcher Lite –...|             1.2.4|4.0.3 and up|    August 1, 2018|8.7M|      Everyone|ART_AND_DESIGN|
|Sketch - Draw & P...|Varies with device|  4.2 and up|      June 8, 2018| 25M|          Teen|ART_AND_DESIGN|
|Pixel Draw - Numb...|               1.1|  4.4 and up|     June 20, 2018|2.8M|      Everyone|ART_AND_DESIGN|
|Paper flowers ins...|               1.0|  2.3 and up|    March 26, 2017|5.6M|      Everyone|ART_AND_DESIGN|
|Smoke Effect Phot.

In [46]:
# count the rows in app dimension
dim_app.count()

10841

In [47]:
# drop the duplicates in App column in app dimension 
dim_app = dim_app.dropDuplicates(["App"])

In [48]:
# count the rows in app dimension
dim_app.count()

9660

In [49]:
# trim the app name for extra spaces
dim_app = dim_app.withColumn("App",trim(dim_app.App))
# replace "(double quotes) with empty char in app name column
dim_app = dim_app.withColumn("App",translate(col("App"),"\"",""))
# cast Last Updated column to date, before it replace 0 with null
dim_app = dim_app.replace(0,None,'Last Updated')\
                    .withColumn("Last Updated",to_date(col("Last Updated"),"MMMM d, yyyy"))
dim_app.show()

+--------------------+------------------+------------------+------------+-------------------+--------------+-------------------+
|                 App|       Current Ver|       Android Ver|Last Updated|               Size|Content Rating|           Category|
+--------------------+------------------+------------------+------------+-------------------+--------------+-------------------+
|i DT Fútbol. Todo...|              0.22|        4.1 and up|  2017-10-07|               3.6M|      Everyone|             SPORTS|
|Alphabet H Passco...|               1.0|        4.1 and up|  2017-12-21|               3.0M|      Everyone|    PERSONALIZATION|
|Eat Fast Prepare ...|             3.6.6|      4.0.3 and up|  2018-06-10|                17M|      Everyone|     FOOD_AND_DRINK|
|Official QR Code®...|Varies with device|Varies with device|  2018-03-23| Varies with device|      Everyone|       PRODUCTIVITY|
|The FN Baby pisto...|Android 3.0 - 2015|        1.6 and up|  2015-09-06|               8.8M|    

In [50]:
dim_app.printSchema()

root
 |-- App: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)
 |-- Last Updated: date (nullable = true)
 |-- Size: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Category: string (nullable = true)



In [51]:
dim_app.show()

+--------------------+------------------+------------------+------------+-------------------+--------------+-------------------+
|                 App|       Current Ver|       Android Ver|Last Updated|               Size|Content Rating|           Category|
+--------------------+------------------+------------------+------------+-------------------+--------------+-------------------+
|i DT Fútbol. Todo...|              0.22|        4.1 and up|  2017-10-07|               3.6M|      Everyone|             SPORTS|
|Alphabet H Passco...|               1.0|        4.1 and up|  2017-12-21|               3.0M|      Everyone|    PERSONALIZATION|
|Eat Fast Prepare ...|             3.6.6|      4.0.3 and up|  2018-06-10|                17M|      Everyone|     FOOD_AND_DRINK|
|Official QR Code®...|Varies with device|Varies with device|  2018-03-23| Varies with device|      Everyone|       PRODUCTIVITY|
|The FN Baby pisto...|Android 3.0 - 2015|        1.6 and up|  2015-09-06|               8.8M|    

In [52]:
# creating a temporary view of 
# Dataframe and storing it into storeTemp
dim_app.createOrReplaceTempView("dimAppTemp")
 
# using the SQL query to get the max and min last updated date
spark.sql("""select max(`Last Updated`),min(`Last Updated`) from dimAppTemp""").show()

+-----------------+-----------------+
|max(Last Updated)|min(Last Updated)|
+-----------------+-----------------+
|       2018-08-08|       2010-05-21|
+-----------------+-----------------+



In [53]:
# convert it to pandas df and add index column in it
pd_dim_app = dim_app.toPandas()
pd_dim_app["App ID"] = pd_dim_app.index+1
#convert to pandas datetime
pd_dim_app['Last Updated'] = pd.to_datetime(pd_dim_app['Last Updated'], errors='coerce')

pd_dim_app

,App,Current Ver,Android Ver,Last Updated,Size,Content Rating,Category,App ID
0,i DT Fútbol. Todos Somos Técnicos.,0.22,4.1 and up,2017-10-07,3.6M,Everyone,SPORTS,1
1,Alphabet H Passcode Lock Screen,1.0,4.1 and up,2017-12-21,3.0M,Everyone,PERSONALIZATION,2
2,Eat Fast Prepare Without Internet,3.6.6,4.0.3 and up,2018-06-10,17M,Everyone,FOOD_AND_DRINK,3
3,Official QR Code® Reader Q,Varies with device,Varies with device,2018-03-23,Varies with device,Everyone,PRODUCTIVITY,4
4,The FN Baby pistol explained,Android 3.0 - 2015,1.6 and up,2015-09-06,8.8M,Everyone,BOOKS_AND_REFERENCE,5
...,...,...,...,...,...,...,...,...
9655,"뽕티비 - 개인방송, 인터넷방송, BJ방송",4.0.7,4.0.3 and up,2018-07-18,59M,Mature 17+,VIDEO_PLAYERS,9656
9656,💎 I'm rich,1.0.0,4.4 and up,2018-03-11,26M,Everyone,LIFESTYLE,9657
9657,"💘 WhatsLov: Smileys of love, stickers and GIF",4.2.4,4.0.3 and up,2018-07-24,18M,Everyone,SOCIAL,9658
9658,📏 Smart Ruler ↔️ cm/inch measuring for homework!,1.0,4.2 and up,2017-10-21,3.2M,Everyone,TOOLS,9659


In [54]:
pd_dim_app.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9660 entries, 0 to 9659
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   App             9660 non-null   object        
 1   Current Ver     9659 non-null   object        
 2   Android Ver     9659 non-null   object        
 3   Last Updated    9657 non-null   datetime64[ns]
 4   Size            9660 non-null   object        
 5   Content Rating  9659 non-null   object        
 6   Category        9660 non-null   object        
 7   App ID          9660 non-null   int64         
dtypes: datetime64[ns](1), int64(1), object(6)
memory usage: 603.9+ KB


In [55]:
# created dim category df by selecting columns from store df
dim_category = store.select(col("Category"))

In [56]:
dim_category.show()
dim_category.count()

+--------------+
|      Category|
+--------------+
|ART_AND_DESIGN|
|ART_AND_DESIGN|
|ART_AND_DESIGN|
|ART_AND_DESIGN|
|ART_AND_DESIGN|
|ART_AND_DESIGN|
|ART_AND_DESIGN|
|ART_AND_DESIGN|
|ART_AND_DESIGN|
|ART_AND_DESIGN|
|ART_AND_DESIGN|
|ART_AND_DESIGN|
|ART_AND_DESIGN|
|ART_AND_DESIGN|
|ART_AND_DESIGN|
|ART_AND_DESIGN|
|ART_AND_DESIGN|
|ART_AND_DESIGN|
|ART_AND_DESIGN|
|ART_AND_DESIGN|
+--------------+
only showing top 20 rows



10841

In [57]:
# drop the duplicates in Category column in category dimension 
dim_category = dim_category.dropDuplicates(["Category"])

In [58]:
dim_category.show()
dim_category.count()

+-------------------+
|           Category|
+-------------------+
|             EVENTS|
|             COMICS|
|             SPORTS|
|            WEATHER|
|      VIDEO_PLAYERS|
|  AUTO_AND_VEHICLES|
|          PARENTING|
|      ENTERTAINMENT|
|    PERSONALIZATION|
| HEALTH_AND_FITNESS|
|   TRAVEL_AND_LOCAL|
|BOOKS_AND_REFERENCE|
|     FOOD_AND_DRINK|
|        PHOTOGRAPHY|
|           BUSINESS|
|             FAMILY|
|           SHOPPING|
|     HOUSE_AND_HOME|
|               GAME|
|               Face|
+-------------------+
only showing top 20 rows



36

In [59]:
# convert it to pandas df and add index column in it
pd_dim_category = dim_category.toPandas()
pd_dim_category["Category ID"] = pd_dim_category.index+1

pd_dim_category

,Category,Category ID
0,EVENTS,1
1,COMICS,2
2,SPORTS,3
3,WEATHER,4
4,VIDEO_PLAYERS,5
5,AUTO_AND_VEHICLES,6
6,PARENTING,7
7,ENTERTAINMENT,8
8,PERSONALIZATION,9
9,HEALTH_AND_FITNESS,10


In [60]:
# created dim genres df by selecting columns from store df
dim_genres = store.select(col("App"),col("Genres"))
dim_genres.show()
dim_genres.count()

+--------------------+--------------------+
|                 App|              Genres|
+--------------------+--------------------+
|Photo Editor & Ca...|        Art & Design|
| Coloring book moana|Art & Design;Pret...|
|U Launcher Lite –...|        Art & Design|
|Sketch - Draw & P...|        Art & Design|
|Pixel Draw - Numb...|Art & Design;Crea...|
|Paper flowers ins...|        Art & Design|
|Smoke Effect Phot...|        Art & Design|
|    Infinite Painter|        Art & Design|
|Garden Coloring Book|        Art & Design|
|Kids Paint Free -...|Art & Design;Crea...|
|Text on Photo - F...|        Art & Design|
|Name Art Photo Ed...|        Art & Design|
|Tattoo Name On My...|        Art & Design|
|Mandala Coloring ...|        Art & Design|
|3D Color Pixel by...|        Art & Design|
|Learn To Draw Kaw...|        Art & Design|
|Photo Designer - ...|        Art & Design|
|350 Diy Room Deco...|        Art & Design|
|FlipaClip - Carto...|        Art & Design|
|        ibis Paint X|        Ar

10841

In [61]:
# drop the duplicates in App column in genres dimension 
dim_genres = dim_genres.dropDuplicates(["App"])
# apply the same transformation for app name as applied earlier
dim_genres = dim_genres.withColumn("App",translate(col("App"),"\"",""))
dim_genres = dim_genres.withColumn("App",trim(dim_genres.App))

dim_genres.show()
dim_genres.count()

+--------------------+-----------------+
|                 App|           Genres|
+--------------------+-----------------+
|i DT Fútbol. Todo...|           Sports|
|Alphabet H Passco...|  Personalization|
|Eat Fast Prepare ...|     Food & Drink|
|Official QR Code®...|     Productivity|
|The FN Baby pisto...|Books & Reference|
|Womens Health Tip...|             Free|
|  Yanosik: antyradar|             Free|
|+Download 4 Insta...|           Social|
|- Free Comics - C...|           Comics|
|                  .R|            Tools|
|              /u/app|    Communication|
|              058.ba| News & Magazines|
|      1. FC Köln App|           Sports|
|10 Best Foods for...| Health & Fitness|
|10 Minutes a Day ...|        Education|
|10 WPM Amateur ha...|    Communication|
|10,000 Quotes DB ...|Books & Reference|
|100 Doors of Revenge|           Puzzle|
|     100+ C Programs|        Education|
|100000+ Messages ...|        Lifestyle|
+--------------------+-----------------+
only showing top

9660

In [62]:
# Split the Genres column with ';' and have a array of values in it
dim_genres = dim_genres.withColumn("Genres",split(trim(dim_genres.Genres),";"))
dim_genres.show()
# Explode the Genres column, having a Row for each value present in the array
dim_genres = dim_genres.withColumn("Genres",explode(dim_genres.Genres))
dim_genres.show()
dim_genres.count()

+--------------------+-------------------+
|                 App|             Genres|
+--------------------+-------------------+
|i DT Fútbol. Todo...|           [Sports]|
|Alphabet H Passco...|  [Personalization]|
|Eat Fast Prepare ...|     [Food & Drink]|
|Official QR Code®...|     [Productivity]|
|The FN Baby pisto...|[Books & Reference]|
|Womens Health Tip...|             [Free]|
|  Yanosik: antyradar|             [Free]|
|+Download 4 Insta...|           [Social]|
|- Free Comics - C...|           [Comics]|
|                  .R|            [Tools]|
|              /u/app|    [Communication]|
|              058.ba| [News & Magazines]|
|      1. FC Köln App|           [Sports]|
|10 Best Foods for...| [Health & Fitness]|
|10 Minutes a Day ...|        [Education]|
|10 WPM Amateur ha...|    [Communication]|
|10,000 Quotes DB ...|[Books & Reference]|
|100 Doors of Revenge|           [Puzzle]|
|     100+ C Programs|        [Education]|
|100000+ Messages ...|        [Lifestyle]|
+----------

10053

In [63]:
# creating a temporary view of 
# Dataframe and storing it into storeTemp
dim_genres.createOrReplaceTempView("dimGenresTemp")
 
# using the SQL query to get the app having more than 1 genres
spark.sql("""select App, Count(*) as genres from dimGenresTemp group by App having genres>1 order by genres DESC""").show()

+--------------------+------+
|                 App|genres|
+--------------------+------+
|Learn the letters...|     2|
|            Q Wunder|     2|
|Toddler Learning ...|     2|
|Happy Fruits Bomb...|     2|
|           Boomerang|     2|
|     Candy Pop Story|     2|
|No.Draw - Colors ...|     2|
|Whoowasit? - Best...|     2|
|Bejeweled Stars: ...|     2|
|Strawberry Shortc...|     2|
|SweetLand — Famil...|     2|
|Dr. Panda Restaur...|     2|
|Flipped Out! - Po...|     2|
|All-in-One Mahjon...|     2|
|Basketball FRVR -...|     2|
|CN Superstar Socc...|     2|
|NBA JAM by EA SPO...|     2|
|Dr. Panda's Swimm...|     2|
|Drawing for Kids ...|     2|
|Batman: Gotham’s ...|     2|
+--------------------+------+
only showing top 20 rows



In [64]:
# convert it to pandas df and add index column in it
pd_dim_genres = dim_genres.toPandas()
pd_dim_genres["Genres ID"] = pd_dim_genres.index+1

pd_dim_genres

,App,Genres,Genres ID
0,i DT Fútbol. Todos Somos Técnicos.,Sports,1
1,Alphabet H Passcode Lock Screen,Personalization,2
2,Eat Fast Prepare Without Internet,Food & Drink,3
3,Official QR Code® Reader Q,Productivity,4
4,The FN Baby pistol explained,Books & Reference,5
...,...,...,...
10048,"뽕티비 - 개인방송, 인터넷방송, BJ방송",Video Players & Editors,10049
10049,💎 I'm rich,Lifestyle,10050
10050,"💘 WhatsLov: Smileys of love, stickers and GIF",Social,10051
10051,📏 Smart Ruler ↔️ cm/inch measuring for homework!,Tools,10052


In [65]:
# create dim date by droping duplicates in dates
dim_date = dim_app.select(col('Last Updated'))
dim_date = dim_date.dropDuplicates(["Last Updated"])

#convert to pandas df and convert it to datetime
pd_dim_date = dim_date.toPandas()
pd_dim_date['Last Updated'] = pd.to_datetime(pd_dim_date['Last Updated'], errors='coerce')

#add columns required to df
pd_dim_date['Day'] = pd_dim_date['Last Updated'].dt.day
pd_dim_date['Month'] = pd_dim_date['Last Updated'].dt.month
pd_dim_date['Year'] = pd_dim_date['Last Updated'].dt.year
pd_dim_date['Quarter'] = pd_dim_date['Last Updated'].dt.quarter
pd_dim_date['Date'] = pd_dim_date['Last Updated']

pd_dim_date.info()

#drop NA rows
pd_dim_date = pd_dim_date.dropna()

#cast rows to int
pd_dim_date['Day'] = pd_dim_date['Day'].astype(int)
pd_dim_date['Month'] = pd_dim_date['Month'].astype(int)
pd_dim_date['Year'] = pd_dim_date['Year'].astype(int)
pd_dim_date['Quarter'] = pd_dim_date['Quarter'].astype(int)

#create Date ID in YYYYMMDD format
pd_dim_date['Date ID'] = pd_dim_date['Date'].astype(str)
pd_dim_date['Date ID'] = pd_dim_date['Date ID'].str.slice(0,4)+pd_dim_date['Date ID'].str.slice(5,7)+pd_dim_date['Date ID'].str.slice(8,10)
pd_dim_date['Date ID'] = pd_dim_date['Date ID'].astype(int)

#selected required columns
pd_dim_date = pd_dim_date[['Date ID','Date','Day','Month','Year','Quarter','Last Updated']]

pd_dim_date

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1378 entries, 0 to 1377
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Last Updated  1377 non-null   datetime64[ns]
 1   Day           1377 non-null   float64       
 2   Month         1377 non-null   float64       
 3   Year          1377 non-null   float64       
 4   Quarter       1377 non-null   float64       
 5   Date          1377 non-null   datetime64[ns]
dtypes: datetime64[ns](2), float64(4)
memory usage: 64.7 KB


,Date ID,Date,Day,Month,Year,Quarter,Last Updated
0,20150519,2015-05-19,19,5,2015,2,2015-05-19
1,20180528,2018-05-28,28,5,2018,2,2018-05-28
2,20170911,2017-09-11,11,9,2017,3,2017-09-11
3,20141112,2014-11-12,12,11,2014,4,2014-11-12
4,20160301,2016-03-01,1,3,2016,1,2016-03-01
...,...,...,...,...,...,...,...
1372,20180203,2018-02-03,3,2,2018,1,2018-02-03
1373,20170813,2017-08-13,13,8,2017,3,2017-08-13
1374,20180310,2018-03-10,10,3,2018,1,2018-03-10
1375,20180107,2018-01-07,7,1,2018,1,2018-01-07


In [66]:
#create fact table using the pd dims
pd_fact_apps = pd_dim_app.merge(pd_dim_category,on="Category")\
                .merge(pd_dim_genres,on="App")\
                .merge(pd_dim_date,on="Last Updated")
#genearte an ID column
pd_fact_apps["ID"] = pd_fact_apps.index+1
#select the required columns
pd_fact_apps = pd_fact_apps[['ID','App ID','Category ID','Date ID']]
#rename the Date ID column to Last Update Date ID
pd_fact_apps.rename({'Date ID':'Last Update Date ID'},axis = 1, inplace=True)
#initialize new columns in fact with random numbers (just for test)
pd_fact_apps['Installs'] = np.random.randint(1000,5000000, size=len(pd_fact_apps))
pd_fact_apps['Reviews'] = np.random.randint(100,50000, size=len(pd_fact_apps))
pd_fact_apps['Ratings'] = 0.0

pd_fact_apps.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10050 entries, 0 to 10049
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   10050 non-null  int64  
 1   App ID               10050 non-null  int64  
 2   Category ID          10050 non-null  int64  
 3   Last Update Date ID  10050 non-null  int32  
 4   Installs             10050 non-null  int32  
 5   Reviews              10050 non-null  int32  
 6   Ratings              10050 non-null  float64
dtypes: float64(1), int32(3), int64(3)
memory usage: 432.0 KB


In [72]:
pd_fact_apps.head()

,ID,App ID,Category ID,Last Update Date ID,Installs,Reviews,Ratings
0,1,1,3,20171007,3084064,9453,0.0
1,2,1012,16,20171007,2903378,8532,0.0
2,3,2036,16,20171007,693781,45772,0.0
3,4,3910,16,20171007,960426,37648,0.0
4,5,8666,19,20171007,667650,26399,0.0


In [68]:
#select the required columns
pd_dim_app = pd_dim_app[['App ID','App','Current Ver','Android Ver','Size','Content Rating','Last Updated']]

pd_dim_app.head()

,App ID,App,Current Ver,Android Ver,Size,Content Rating,Last Updated
0,1,i DT Fútbol. Todos Somos Técnicos.,0.22,4.1 and up,3.6M,Everyone,2017-10-07
1,2,Alphabet H Passcode Lock Screen,1.0,4.1 and up,3.0M,Everyone,2017-12-21
2,3,Eat Fast Prepare Without Internet,3.6.6,4.0.3 and up,17M,Everyone,2018-06-10
3,4,Official QR Code® Reader Q,Varies with device,Varies with device,Varies with device,Everyone,2018-03-23
4,5,The FN Baby pistol explained,Android 3.0 - 2015,1.6 and up,8.8M,Everyone,2015-09-06


In [69]:
pd_dim_category.head()

,Category,Category ID
0,EVENTS,1
1,COMICS,2
2,SPORTS,3
3,WEATHER,4
4,VIDEO_PLAYERS,5


In [70]:
#merge with dim app to get the App ID and select the required columns
pd_dim_genres = pd_dim_genres.merge(pd_dim_app,on="App")\
                [['Genres ID','App ID','Genres']]

pd_dim_genres.head()

,Genres ID,App ID,Genres
0,1,1,Sports
1,2,2,Personalization
2,3,3,Food & Drink
3,4,4,Productivity
4,5,5,Books & Reference


In [71]:
#select the required columns
pd_dim_date = pd_dim_date[['Date ID','Date','Day','Month','Year','Quarter']]

pd_dim_date.head()

,Date ID,Date,Day,Month,Year,Quarter
0,20150519,2015-05-19,19,5,2015,2
1,20180528,2018-05-28,28,5,2018,2
2,20170911,2017-09-11,11,9,2017,3
3,20141112,2014-11-12,12,11,2014,4
4,20160301,2016-03-01,1,3,2016,1


In [86]:
reviews.show()
reviews.printSchema()

+--------------------+--------------------+--------------------+-------------------+----------------------+
|                 App|   Translated_Review|           Sentiment| Sentiment_Polarity|Sentiment_Subjectivity|
+--------------------+--------------------+--------------------+-------------------+----------------------+
|10 Best Foods for...|"I like eat delic...| also ""Best Befo...|           Positive|                   1.0|
|10 Best Foods for...|This help eating ...|            Positive|               0.25|   0.28846153846153844|
|10 Best Foods for...|                 nan|                 nan|                nan|                   nan|
|10 Best Foods for...|Works great espec...|            Positive|                0.4|                 0.875|
|10 Best Foods for...|        Best idea us|            Positive|                1.0|                   0.3|
|10 Best Foods for...|            Best way|            Positive|                1.0|                   0.3|
|10 Best Foods for...|      

In [94]:
#create dim reviews table
dim_reviews = reviews.select(col('App'),col('Translated_Review'))

# apply the same transformation for app name as applied earlier
dim_reviews = dim_reviews.withColumn("App",translate(col("App"),"\"",""))
dim_reviews = dim_reviews.withColumn("App",trim(dim_reviews.App))
dim_reviews = dim_reviews.withColumnRenamed('Translated_Review','Review')

pd_dim_reviews = dim_reviews.toPandas()
#generate index ID
pd_dim_reviews["Review ID"] = pd_dim_reviews.index+1
#initialize new columns in fact with random numbers (just for test)
pd_dim_reviews['Sentiment Analysis'] = np.random.uniform(-5.0,5.0,len(pd_dim_reviews))
pd_dim_reviews['Ratings'] = np.random.randint(1,5, size=len(pd_dim_reviews))

#merge with dim app to get the App ID and select the required columns
pd_dim_reviews = pd_dim_reviews.merge(pd_dim_app,on="App")\
                [['Review ID','App ID','Review','Ratings','Sentiment Analysis']]

pd_dim_reviews.head()

,Review ID,App ID,Review,Ratings,Sentiment Analysis
0,1,14,"""I like eat delicious food. That's I'm cooking...",4,-2.661034
1,2,14,This help eating healthy exercise regular basis,1,3.792019
2,3,14,nan,2,2.984275
3,4,14,Works great especially going grocery store,1,-0.620018
4,5,14,Best idea us,3,-4.704927


In [95]:
#create dim installs table
dim_installs = reviews.select(col('App'))

# apply the same transformation for app name as applied earlier
dim_installs = dim_installs.withColumn("App",translate(col("App"),"\"",""))
dim_installs = dim_installs.withColumn("App",trim(dim_installs.App))

pd_dim_installs = dim_installs.toPandas()
#generate index ID
pd_dim_installs["Install ID"] = pd_dim_installs.index+1
#initialize new columns in fact with random numbers (just for test)
pd_dim_installs['User'] = np.random.randint(1,5000000, size=len(pd_dim_installs))
pd_dim_installs['Device'] = np.random.randint(1,70000000, size=len(pd_dim_installs))

#merge with dim app to get the App ID and select the required columns
pd_dim_installs = pd_dim_installs.merge(pd_dim_app,on="App")\
                [['Install ID','App ID','User','Device']]

pd_dim_installs.head()

,Install ID,App ID,User,Device
0,1,14,3.615815e+05,31159372
1,2,14,3.630250e+06,5741148
2,3,14,1.610681e+06,11537040
3,4,14,2.785785e+06,39819789
4,5,14,4.083932e+06,9530392
